<a href="https://colab.research.google.com/github/antonychackotc/machine-learning-supervised/blob/main/Activity_5(jan_12)_manually_find_classifier_regressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **upload a file then train model on uploaded file data its Manualy select Linear Reggressor or decession tree Regressor or logistic Regressor or Decession tree classifier check file character one by one**

In [1]:
!pip install -q streamlit
!pip install -q pyngrok
!pip install -q localtunnel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.5 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement localtunnel (from versions: none)
ERROR: No matching distribution found for localtunnel


In [ ]:
%%writefile app1.py

In [13]:
%%writefile app1.py

import pandas as pd
import numpy as np
import streamlit as st
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, r2_score
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR, SVC
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import PolynomialFeatures

# Function to evaluate models for regression
def evaluate_regression(model, X_test, y_test):
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    return r2 * 100

# Function to evaluate models for classification
def evaluate_classification(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy * 100

# Function to apply a single model
def apply_single_model(X_train, X_test, y_train, y_test, model_name, task_type):
    model_map = {
        "Linear Regression": LinearRegression(),
        "Polynomial Regression": PolynomialFeatures(degree=2),
        "Ridge Regression": Ridge(),
        "Lasso Regression": Lasso(),
        "ElasticNet Regression": ElasticNet(),
        "SVM Regression": SVR(),
        "Decision Tree Regressor": DecisionTreeRegressor(),
        "Random Forest Regressor": RandomForestRegressor(),
        "Boosting Regressor": GradientBoostingRegressor(),
        "KNN Regressor": KNeighborsRegressor(),
        "Logistic Regression": LogisticRegression(),
        "SVM Classifier": SVC(),
        "Decision Tree Classifier": DecisionTreeClassifier(),
        "Random Forest Classifier": RandomForestClassifier(),
        "Boosting Classifier": GradientBoostingClassifier(),
        "KNN Classifier": KNeighborsClassifier(),
        "Naive Bayes Classifier": GaussianNB(),
    }

    if model_name == "Polynomial Regression":
        poly = PolynomialFeatures(degree=2)
        X_train_poly = poly.fit_transform(X_train)
        X_test_poly = poly.transform(X_test)
        lin_reg = LinearRegression()
        lin_reg.fit(X_train_poly, y_train)
        score = evaluate_regression(lin_reg, X_test_poly, y_test)
        trained_model = lin_reg
    else:
        model = model_map[model_name]
        model.fit(X_train, y_train)
        if task_type == 'regression':
            score = evaluate_regression(model, X_test, y_test)
        else:
            score = evaluate_classification(model, X_test, y_test)
        trained_model = model

    return trained_model, score

# Streamlit UI
st.title("Automated Model Selection with Future Prediction")

# Tabs for better navigation
tabs = st.tabs(["Data Analysis", "Model Evaluation", "Future Prediction"])

with tabs[0]:  # Data Analysis Tab
    st.header("Data Analysis")
    uploaded_file = st.file_uploader("Upload your dataset (CSV format):", type=["csv"])
    if uploaded_file is not None:
        data = pd.read_csv(uploaded_file)
        st.write("Dataset Preview:", data.head())

        y_col = st.selectbox("Select the target column:", data.columns)
        if y_col:
            y = data[y_col]
            unique_values = y.nunique()

            # Infer task type
            if y.dtype in [np.int64, np.float64]:
                if unique_values > 20:
                    inferred_task_type = 'regression'
                else:
                    inferred_task_type = 'classification'
            else:
                inferred_task_type = 'classification'

            st.write(f"Inferred Task Type: **{inferred_task_type}**")
            if inferred_task_type == 'regression':
                st.write("Suggested Model: **Linear Regression** or **Random Forest Regressor**")
            else:
                st.write("Suggested Model: **Random Forest Classifier** or **Logistic Regression**")

with tabs[1]:  # Model Evaluation Tab
    st.header("Model Evaluation")
    if uploaded_file is not None:
        X_cols = st.multiselect("Select feature columns:", [col for col in data.columns if col != y_col])
        if not X_cols:
            st.warning("Please select at least one feature column.")
        else:
            X = data[X_cols]
            y = data[y_col]

            task_type = st.radio("Confirm Task Type:", ["regression", "classification"])
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

            model_list = [
                "Linear Regression", "Polynomial Regression", "Ridge Regression", "Lasso Regression",
                "ElasticNet Regression", "SVM Regression", "Decision Tree Regressor",
                "Random Forest Regressor", "Boosting Regressor", "KNN Regressor"
            ] if task_type == 'regression' else [
                "Logistic Regression", "SVM Classifier", "Decision Tree Classifier",
                "Random Forest Classifier", "Boosting Classifier", "KNN Classifier",
                "Naive Bayes Classifier"
            ]

            selected_model = st.selectbox("Select a model to evaluate:", model_list)
            if st.button("Run Selected Model"):
                trained_model, score = apply_single_model(X_train, X_test, y_train, y_test, selected_model, task_type)
                st.session_state["trained_model"] = trained_model  # Save the trained model to session state
                st.session_state["X_cols"] = X_cols  # Save selected feature columns
                st.success(f"Model {selected_model} achieved a score of {score:.2f}%.")

with tabs[2]:  # Future Prediction Tab
    st.header("Future Prediction")

    # Check if a dataset is uploaded and a model is trained
    if uploaded_file is not None and "trained_model" in st.session_state:
        st.write("Provide data for prediction using the existing features.")

        # Create a form for entering input data for prediction
        input_data = {col: st.number_input(f"Enter value for {col}:", value=0.0) for col in st.session_state["X_cols"]}

        # Convert input data to a DataFrame
        input_df = pd.DataFrame([input_data])
        st.write("Input Data Preview:")
        st.write(input_df)

        # Button to generate predictions
        if st.button("Generate Prediction"):
            try:
                # Retrieve the trained model from session state
                trained_model = st.session_state["trained_model"]
                predictions = trained_model.predict(input_df)
                st.write("Predictions:", predictions)
            except Exception as e:
                st.error(f"An error occurred during prediction: {e}")
    else:
        st.warning("Please upload a dataset and evaluate a model in the 'Model Evaluation' tab before generating predictions.")


Overwriting app1.py


In [3]:
from pyngrok import ngrok

# Replace 'YOUR_AUTHTOKEN' with your actual ngrok authtoken
ngrok.set_auth_token("2rI2XurhgC2fxlYDtteHntWpCJf_5b1kDx2SLmwgq8GukDEyc")

# Run the Streamlit app in the background
!streamlit run app1.py &>/dev/null&

# Create a public URL using ngrok
try:
    public_url = ngrok.connect(8501)
    print(f"Streamlit app is running at {public_url}")
except Exception as e:
    print(f"Error: {e}")
    print("Trying to run with localtunnel")
    !streamlit run app1.py &>/content/logs.txt & npx localtunnel --port 8501

Streamlit app is running at NgrokTunnel: "https://1077-34-148-88-174.ngrok-free.app" -> "http://localhost:8501"
